# Working with local coordinates 

<img align="right" src="https://movingpandas.github.io/movingpandas/assets/img/movingpandas.png">

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/movingpandas/movingpandas/main?filepath=tutorials/5-local-coordinates.ipynb)

**<p style="color:#e31883">This notebook demonstrates the current development version of MovingPandas.</p>**

For tutorials using the latest release visit https://github.com/movingpandas/movingpandas-examples.



This tutorial uses data extracted from video footage of a soccer game that was published in https://github.com/Friends-of-Tracking-Data-FoTD/Last-Row


In [ ]:
import urllib
import os
import pandas as pd
import geopandas as gpd
from geopandas import GeoDataFrame, read_file
from shapely.geometry import Point, LineString, Polygon
from datetime import datetime, timedelta
from matplotlib import pyplot as plt
import holoviews as hv

import sys

sys.path.append("..")
import movingpandas as mpd

mpd.show_versions()

import warnings

warnings.simplefilter("ignore")

In [ ]:
from os.path import exists
from urllib.request import urlretrieve


def get_df_from_gh_url(url):
    file = f"soccer.csv"
    if not exists(file):
        urlretrieve(url, file)
    return pd.read_csv(file)

In [ ]:
input_file = "https://raw.githubusercontent.com/anitagraser/Last-Row/master/datasets/positional_data/liverpool_2019.csv"
df = get_df_from_gh_url(input_file)
df.drop(columns=["Unnamed: 0"], inplace=True)

In [ ]:
plays = list(df.play.unique())


def to_timestamp(row):
    # plays to date
    day = plays.index(row.play) + 1
    start_time = datetime(2019, 1, day, 12, 0, 0)
    # frames to time
    td = timedelta(milliseconds=1000 / 20 * row.frame)
    return start_time + td


# frame: the frame number for the current location. Data provided has 20 frames per second
df["time"] = df.apply(to_timestamp, axis=1)
df.set_index("time", inplace=True)

# the preferred size for many professional teams' stadiums is 105 by 68 metres, according to https://en.wikipedia.org/wiki/Football_pitch
pitch_length = 105
pitch_width = 68
df.x = df.x / 100 * pitch_length
df.y = df.y / 100 * pitch_width

df

In [ ]:
%%time
CRS = None
# df['player'] = df['team']+df['player'].astype(str)
tc = mpd.TrajectoryCollection(df, "player", x="x", y="y", crs=CRS)
mpd.TemporalSplitter(tc).split(mode="day")
print(f"Finished creating {len(tc)} trajectories")

In [ ]:
from hvplot import pandas

pitch = Polygon(
    [(0, 0), (0, pitch_width), (pitch_length, pitch_width), (pitch_length, 0), (0, 0)]
)
plotted_pitch = GeoDataFrame(
    pd.DataFrame([{"geometry": pitch, "id": 1}]), crs=CRS
).hvplot(color="white", alpha=0.5)

In [ ]:
PLAY = 2
title = f"Play {PLAY} {plays[PLAY]}"
play_trajs = tc.filter("play", plays[PLAY])
play_trajs

In [ ]:
play_trajs.plot(column="team", colormap={"attack": "hotpink", "defense": "turquoise"})

In [ ]:
type(play_trajs)

In [ ]:
generalized = mpd.MinTimeDeltaGeneralizer(play_trajs).generalize(
    tolerance=timedelta(seconds=0.5)
)

In [ ]:
hvplot_defaults = {
    "line_width": 5,
    "frame_height": 350,
    "frame_width": 500,
    "colorbar": True,
    "tiles": None,
    "geo": False,
}
generalized.hvplot(
    title=title,
    c="speed",
    hover_cols=["speed", "player"],
    cmap="Plasma",
    **hvplot_defaults
)

In [ ]:
(
    plotted_pitch
    * generalized.hvplot(title=title, hover_cols=["player"], **hvplot_defaults)
)

In [ ]:
(
    plotted_pitch
    * generalized.hvplot(title=title, c="team", hover_cols=["team"], **hvplot_defaults)
)

In [ ]:
pitch_img = hv.RGB.load_image(
    f"./data/soccer_field.png", bounds=(0, 0, pitch_length, pitch_width)
)
(
    pitch_img
    * generalized.hvplot(
        title=title,
        c="team",
        colormap={"attack": "limegreen", "defense": "purple"},
        hover_cols=["player"],
        **hvplot_defaults,
    )
    * generalized.get_start_locations().hvplot(label="start", color="orange")
)

## Continue exploring MovingPandas

1. [Getting started](1-getting-started.ipynb)
1. [Handling trajectory data files (reading & writing)](2-reading-data-from-files.ipynb)
1. [TrajectoryCollection aggregation (flow maps)](3-generalization-and-aggregation.ipynb)
1. [Stop detection](4-stop-detection.ipynb)
1. [Working with local coordinates](5-local-coordinates.ipynb)
1. [Computing trajectory metrics](6-trajectory-metrics.ipynb)
1. [Multithreading](7-multithreading.ipynb)
1. [OGC Moving Features](8-ogc-moving-features.ipynb)